In [1]:
import requests
import urllib.request, urllib.parse, urllib.error
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

Add a flag for hate speech detected from either the
https://www.glaad.org/hate-speech-listing and the ADL https://www.adl.org/resources/hate-symbols/search so two features in total. Compare the text of the post with the filter lists and flag the post as containing that speech or not.

In [2]:
url = 'https://www.adl.org/resources/hate-symbols/search?keywords=&sort_by=title&page=0'

In [3]:
def API_call_hate_speech(url):

    hate_speech_list = []

    # Make a request to the web page
    html = urllib.request.urlopen(url).read()

    # Parse the HTML content of the page
    soup = BeautifulSoup(html, 'html.parser')

    #  <span class="field field--name-title field--type-string field--label-hidden">33/6</span>
    # Find all div blocks that have the class "global-search__row"
    hate_speech = soup.find_all('span', {'class': 'field field--name-title field--type-string field--label-hidden'})

    hate_speech = list(hate_speech)

    for speech in hate_speech:
        hate_speech_list.append(speech.text)

    # print("Length:", len(hate_speech_list))

    return hate_speech_list

In [4]:
hateSpeechList = []

for i in range(11):

    url = 'https://www.adl.org/resources/hate-symbols/search?keywords=&sort_by=title&page=' + str(i)
    # print("***** Page:", i+1)
    hateSpeechList = hateSpeechList + API_call_hate_speech(url)

In [5]:
for i in range(len(hateSpeechList)):

    if i == 5:
        hateSpeechList[i] = "13/52"
        hateSpeechList.append("13/90")
    elif i == 14: 
        hateSpeechList[i] = "23"
    elif i == 26: 
        hateSpeechList[i] = "88"
    elif i == 33: 
        hateSpeechList[i] = "American Identity Movement"
        hateSpeechList.append("AIM")
    elif i == 41 or i == 48 or i == 49 or i == 80 or i == 146 or i == 177 or i == 197 or i == 200: 
        hateSpeechList.pop(i)
    elif i == 62: 
        hateSpeechList[i] = "Bowlcut"
        hateSpeechList.append("Dylann Roof")
    elif i == 92: 
        hateSpeechList[i] = "hammerskin"
    elif i == 98: 
        hateSpeechList[i] = "Hitler salute"
    elif i == 113: 
        hateSpeechList[i] = "Ku Klux Klan"
    elif i == 156: 
        hateSpeechList[i] = "Runic Writing"
    elif i == 168: 
        hateSpeechList[i] = "South African Flag"
        hateSpeechList.append("Apartheid Era")
    elif i == 172: 
        hateSpeechList[i] = "SS"
    elif i == 180: 
        hateSpeechList[i] = "The Goyim Know"
        hateSpeechList.append("Shut It Down")
    elif i == 188: 
        hateSpeechList[i] = "Trollface"
    elif i == 201: 
        hateSpeechList[i] = "WAR"
        hateSpeechList.append("Arkansas prison gang")
    elif i == 206: 
        hateSpeechList[i] = "white power"


In [6]:
# for i in range(len(hateSpeechList)):
#     print(i, "-", hateSpeechList[i])

# Extracting Narrative

In [7]:
df_master_dataset = pd.read_csv("../Master_Dataset_Raw.csv")

In [13]:
# Define a function to extract the text from an HTML string
def extract_text(html):
    # print('ran')
    try: 
        soup = BeautifulSoup(html, 'html.parser')
        text = soup.get_text()
    except:
        # print("ran except")
        text = html

    return text

# Apply the function to each row of the DataFrame and create a new column with the result
df_master_dataset['Narrative'] = df_master_dataset['Narrative'].apply(extract_text)

/Users/toddgavin/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/Users/toddgavin/opt/anaconda3/lib/python3.9/site-packages/bs4/__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [15]:
df_master_dataset

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Interest
0,156581,STY1666112066,86625,USR1662300553,NaN,12,Qatar Airways cuts flights to accommodate FIFA...,Qatar Airways has withdrawn flights from 18 de...,https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 19:39:13,"Technology, History, Food, Entertainment, Spor..."
1,156580,STY1666111954,119236,USR1665943314,NaN,18,Positive thinking does not necessarily mean av...,Positive thinking does not necessarily mean av...,https://image.pixstory.com/Pixstory-image-1666...,2022-10-16 23:31:54,"Technology, History, Food, Entertainment, Spor..."
2,156579,STY1666111923,86566,USR1662294899,NaN,12,FIFA negotiating 'initiatives' for Qatar migra...,FIFA's president sought Monday to dispell ling...,https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 18:04:59,"Technology, History, Food, Entertainment, Spor..."
3,156578,STY1666111798,86655,USR1662303432,NaN,12,Qatar ready for the World Cup in just over a m...,Ticket sales for the FIFA World Cup Qatar 2022...,https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 20:27:12,"Technology, History, Food, Entertainment, Spor..."
4,156577,STY1666111463,9221,USR1632727815,male,23,"Froyo always makes everyone happy, make this h...","\nAlmost all of us is a fan of ice creams, fro...",https://image.pixstory.com/Pixstory-image-1666...,2021-09-27 13:00:15,"Science, Inequality, Education, Technology, He..."
...,...,...,...,...,...,...,...,...,...,...,...
94995,95021,STY1650382324,22770,USR1642425119,male,40,Ronaldo salta la trasferta di Liverpool,Il Manchester United ha annunciato l'assenza d...,https://image.pixstory.com/Pixstory-image-1650...,2022-01-17 18:41:59,"calcio, premier league, cristiano ronaldo, man..."
94996,95020,STY1650381837,32577,USR1650381467,NaN,15,Name-vishal\nFather name-Amod\nClass-7\nAge-13...,Earthday,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 20:47:47,chintan
94997,95019,STY1650381574,32498,USR1650370047,NaN,15,Earth day,Name Muskan \nClass 8\nSex f,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 17:37:27,chintan
94998,95018,STY1650380690,32574,USR1650380153,NaN,12,Name-puneet\nclass-6th\nfather name-Ramkishan\...,Earthday,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 20:25:53,chintan


# Classify Hate Speech

In [16]:
df_master_dataset['Narrative'] = df_master_dataset['Narrative'].astype(str)

In [51]:
df_master_dataset['Hate Speech Flag'] = ""
df_master_dataset['Hate Speech Detected'] = ""

In [60]:
def classify_text(text, hateSpeechList):

    for word in hateSpeechList:
        if word in text:
            # print("hate")
            return 'Hate Speech Detected', word
        else:
            # print("no hate")
            return 'Not Detected', 'N/A'

In [62]:
# for index, row in df_master_dataset.iterrows():

#     narrative = row["Narrative"]
#     # print(narrative)

#     row["Hate Speech Flag"] = classify_text(narrative, hateSpeechList)
#     # print(speech)

#     # row["Hate Speech Flag"] = speech

#     print(row['Hate Speech Flag'])

for index, row in df_master_dataset.iterrows():
    narrative = row["Narrative"]
    classification, word = classify_text(narrative, hateSpeechList)
    df_master_dataset.loc[index, "Hate Speech Flag"] = classification
    df_master_dataset.loc[index, "Hate Speech Detected"] = word

    # if classification == "Contains Hate Speech":
        # print('**********')
        # print(f"Index: {index}, Word: {word}, Narrative: {narrative}")


In [63]:
df_master_dataset

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Interest,Hate Speech Flag,Hate Speech Detected
0,156581,STY1666112066,86625,USR1662300553,NaN,12,Qatar Airways cuts flights to accommodate FIFA...,Qatar Airways has withdrawn flights from 18 de...,https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 19:39:13,"Technology, History, Food, Entertainment, Spor...",Not Detected,N/A
1,156580,STY1666111954,119236,USR1665943314,NaN,18,Positive thinking does not necessarily mean av...,Positive thinking does not necessarily mean av...,https://image.pixstory.com/Pixstory-image-1666...,2022-10-16 23:31:54,"Technology, History, Food, Entertainment, Spor...",Not Detected,N/A
2,156579,STY1666111923,86566,USR1662294899,NaN,12,FIFA negotiating 'initiatives' for Qatar migra...,FIFA's president sought Monday to dispell ling...,https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 18:04:59,"Technology, History, Food, Entertainment, Spor...",Not Detected,N/A
3,156578,STY1666111798,86655,USR1662303432,NaN,12,Qatar ready for the World Cup in just over a m...,Ticket sales for the FIFA World Cup Qatar 2022...,https://image.pixstory.com/Pixstory-image-1666...,2022-09-04 20:27:12,"Technology, History, Food, Entertainment, Spor...",Not Detected,N/A
4,156577,STY1666111463,9221,USR1632727815,male,23,"Froyo always makes everyone happy, make this h...","\nAlmost all of us is a fan of ice creams, fro...",https://image.pixstory.com/Pixstory-image-1666...,2021-09-27 13:00:15,"Science, Inequality, Education, Technology, He...",Not Detected,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,95021,STY1650382324,22770,USR1642425119,male,40,Ronaldo salta la trasferta di Liverpool,Il Manchester United ha annunciato l'assenza d...,https://image.pixstory.com/Pixstory-image-1650...,2022-01-17 18:41:59,"calcio, premier league, cristiano ronaldo, man...",Not Detected,N/A
94996,95020,STY1650381837,32577,USR1650381467,NaN,15,Name-vishal\nFather name-Amod\nClass-7\nAge-13...,Earthday,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 20:47:47,chintan,Not Detected,N/A
94997,95019,STY1650381574,32498,USR1650370047,NaN,15,Earth day,Name Muskan \nClass 8\nSex f,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 17:37:27,chintan,Not Detected,N/A
94998,95018,STY1650380690,32574,USR1650380153,NaN,12,Name-puneet\nclass-6th\nfather name-Ramkishan\...,Earthday,https://image.pixstory.com/Pixstory-image-1650...,2022-04-19 20:25:53,chintan,Not Detected,N/A


In [64]:
df_master_dataset.to_csv('Master_Dataset_Raw_HateSpeech.csv', index=False)